<h1>Actividad Clasificador de topicos</h1>
<h2>Martes 10 de Agosto</h2>
<ol>
<li>Elegir dos topicos</li>
<li>Obtener al menos 50 frases para cada topico</li>
<li>Definir un set con almenos 25 frases de cada tópico</li>
<li>Definir el código para entrenar un modelo de SGD o Naive Bayes</li>
</ol>



In [375]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_array
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from collections import defaultdict
import csv, sys

In [376]:
canciones_df=pd.read_csv('data/Canciones Romanticas.csv')
documentacion_df=pd.read_csv('data/Lenguajes de programacion.csv')
canciones_df=canciones_df.iloc[:, 0].tolist()
documentacion_df=documentacion_df.iloc[:, 0].tolist()

canciones_df
documentacion_df

['The break statement exits from a loop or switch statement.',
 'The continue statement skips the remaining code in a loop iteration and proceeds to the next iteration.',
 'The for loop executes a block of code a specified number of times.',
 'The foreach loop iterates over a collection of items.',
 'The while loop executes a block of code as long as a specified condition is true.',
 'The do loop executes a block of code once then repeats the execution as long as a specified condition is true.',
 'The try catch statement is used for exception handling.',
 'The interface keyword defines a contract that classes must follow.',
 'The new keyword creates an instance of a class or struct.',
 'The return statement exits a method and optionally returns a value.',
 'The base keyword refers to the base class of the current instance.',
 'The this keyword refers to the current instance of a class.',
 'The params keyword allows a method to accept a variable number of arguments.',
 'The null keyword

In [377]:
canciones_df = list(map(str.lower,canciones_df))
documentacion_df = list(map(str.lower,documentacion_df))

In [378]:
canciones_df

['trying to hold on to your high-class love',
 'you stick around now it may show',
 'before i say another word let me tell you i love you',
 "you've got my head spinning no kidding i can't pin you down",
 'my heart was wrapped up in clover',
 'wise men say only fools rush in',
 'i hate to turn up out of the blue uninvited',
 'and i hope life will treat you kind',
 'are you somewhere feeling lonely or is someone loving you?',
 'what can make me feel this way?',
 'he can never want some other girl',
 'if you leave me now you´ll take away the biggest part of me',
 "i'll be the moon when the sun goes down",
 'love your curves and all your edges',
 'sometimes it lasts in love but sometimes it hurts instead',
 "that i should've bought you flowers and held your hand",
 'true love breathes salvation back into me',
 'and tell you time and time again how much i care',
 "i've got all the riches baby one man can claim",
 'it started out as a feeling which then grew into a hope',
 "that you found a

In [379]:
categories = ['canciones', 'documentacion']

In [380]:
# Join individual documents to calculate cosine similarity
canciones = ' '.join(canciones_df)
documentacion = ' '.join(documentacion_df)
documents = [canciones,documentacion]
count_vectorizer = CountVectorizer(stop_words="english")
#count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vectorizer.get_feature_names_out(),
   index=["canciones", "documentacion"],
)
print(df)
print(cosine_similarity(df, df))

               accept  allows  arguments  assignment  attempts  away  baby  \
canciones           0       0          0           0         0     1     1   
documentacion       1       2          1           1         1     0     0   

               base  biggest  block  ...  variable  ve  want  way  weeping  \
canciones         0        1      0  ...         0   3     1    1        1   
documentacion     3        0      4  ...         2   0     0    0        0   

               willing  willow  wise  word  wrapped  
canciones            1       1     1     1        1  
documentacion        0       0     0     0        0  

[2 rows x 168 columns]
[[1.         0.03147546]
 [0.03147546 1.        ]]


In [381]:
df.shape

(2, 168)

Identify common tokens and resolve

the first approach, delete all, is not right; delete those elements that are less frequent. Example: paper [105 0 5] delete for segment 15 and 32, if the number is the same, delete the word is all bags of words.

There are 201 terms.

Create three lists to keep track of which tokens will be erased for each segment

In [382]:
i = 0
commonTerms = []
cancionesdelete = []
documentaciondelete = []
for (columnName, columnData) in df.items():
    nOcurrences = sum(x != 0 for x in columnData.values)
    if nOcurrences > 1:
      i = i + 1
      commonTerms.append(columnName) # keep track of all the common terms
      print('Token : ', columnName)
      print('Token Frequency : ', columnData.values)
      print('Token Repetition', sum(x != 0 for x in columnData.values))
      cancionescount = columnData.values[0]
      documentacioncount = columnData.values[1]
      print('Canciones Token count ', cancionescount)
      print('Documentacion Token count ', documentacioncount)
      # Less frequent tokens are deleted from the bag of words
      if (cancionescount > documentacioncount) and (cancionescount > documentacioncount):
        print('Canciones wins')
        documentaciondelete.append(columnName)
      elif (documentacioncount > cancionescount) and (documentacioncount > cancionescount):
        print('Documentacion wins')
        cancionesdelete.append(columnName)
      else: # there is a tie, the tokens is destroyed
        cancionesdelete.append(columnName)
        documentaciondelete.append(columnName)
print('Total terms ', i)

Token :  class
Token Frequency :  [1 3]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  3
Documentacion wins
Token :  hand
Token Frequency :  [1 1]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  1
Token :  time
Token Frequency :  [2 1]
Token Repetition 2
Canciones Token count  2
Documentacion Token count  1
Canciones wins
Token :  true
Token Frequency :  [1 2]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  2
Documentacion wins
Token :  try
Token Frequency :  [1 1]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  1
Total terms  5


In [383]:
cancionesc = []
for string in canciones_df:
  print(string)
  cancionesc.append(' '.join(i for i in string.split() if i not in cancionesdelete))

trying to hold on to your high-class love
you stick around now it may show
before i say another word let me tell you i love you
you've got my head spinning no kidding i can't pin you down
my heart was wrapped up in clover
wise men say only fools rush in
i hate to turn up out of the blue uninvited
and i hope life will treat you kind
are you somewhere feeling lonely or is someone loving you?
what can make me feel this way?
he can never want some other girl
if you leave me now you´ll take away the biggest part of me
i'll be the moon when the sun goes down
love your curves and all your edges
sometimes it lasts in love but sometimes it hurts instead
that i should've bought you flowers and held your hand
true love breathes salvation back into me
and tell you time and time again how much i care
i've got all the riches baby one man can claim
it started out as a feeling which then grew into a hope
that you found a girl and you're married now
i'll be the weeping willow drowning in my tears
and i

In [384]:
documentacionc = []
for string in documentacion_df:
  print(string)
  cancionesc.append(' '.join(i for i in string.split() if i not in documentaciondelete))

the break statement exits from a loop or switch statement.
the continue statement skips the remaining code in a loop iteration and proceeds to the next iteration.
the for loop executes a block of code a specified number of times.
the foreach loop iterates over a collection of items.
the while loop executes a block of code as long as a specified condition is true.
the do loop executes a block of code once then repeats the execution as long as a specified condition is true.
the try catch statement is used for exception handling.
the interface keyword defines a contract that classes must follow.
the new keyword creates an instance of a class or struct.
the return statement exits a method and optionally returns a value.
the base keyword refers to the base class of the current instance.
the this keyword refers to the current instance of a class.
the params keyword allows a method to accept a variable number of arguments.
the null keyword represents a null reference.
the is keyword checks if

In [385]:
# Join individual documents to calculate cosine similarity after cleansing
# Reduction of the cosine similarity
canciones = ' '.join(cancionesc)
documentacion = ' '.join(documentacionc)
documents = [canciones, documentacion]
count_vectorizer = CountVectorizer(stop_words="english")
#count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vectorizer.get_feature_names_out(),
   index=["canciones", "documentacion"],
)
print(df)
print(cosine_similarity(df, df))

               accept  allows  arguments  assignment  attempts  away  baby  \
canciones           1       2          1           1         1     1     1   
documentacion       0       0          0           0         0     0     0   

               base  biggest  block  ...  variable  ve  want  way  weeping  \
canciones         3        1      4  ...         2   3     1    1        1   
documentacion     0        0      0  ...         0   0     0    0        0   

               willing  willow  wise  word  wrapped  
canciones            1       1     1     1        1  
documentacion        0       0     0     0        0  

[2 rows x 167 columns]
[[1. 0.]
 [0. 0.]]


In [386]:
fullTrainingDatac = cancionesc + documentacionc
cancionesvaluesc = [1] * len(cancionesc)
documentacionvaluesc = [2] * len(documentacionc)
fullDataClassificationc = cancionesvaluesc + documentacionvaluesc

count_vect = CountVectorizer(stop_words='english')
X_products_countsc = count_vect.fit_transform(fullTrainingDatac)
X_products_countsc.shape

(48, 167)

In [387]:
fullTrainingData = canciones_df + documentacion_df

In [388]:
cancionesvalues = [1] * len(canciones_df)
documentacionvalues = [2] * len(documentacion_df)
fullDataClassification = cancionesvalues + documentacionvalues

In [389]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
X_products_counts = count_vect.fit_transform(fullTrainingData)
X_products_counts.shape

(48, 168)

These are the term matrices

In [390]:
doc_term_matrix = X_products_counts.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vect.get_feature_names_out()
)
print(df)
print(cosine_similarity(df, df))

    accept  allows  arguments  assignment  attempts  away  baby  base  \
0        0       0          0           0         0     0     0     0   
1        0       0          0           0         0     0     0     0   
2        0       0          0           0         0     0     0     0   
3        0       0          0           0         0     0     0     0   
4        0       0          0           0         0     0     0     0   
5        0       0          0           0         0     0     0     0   
6        0       0          0           0         0     0     0     0   
7        0       0          0           0         0     0     0     0   
8        0       0          0           0         0     0     0     0   
9        0       0          0           0         0     0     0     0   
10       0       0          0           0         0     0     0     0   
11       0       0          0           0         0     1     0     0   
12       0       0          0           0         0

Create Inverse Matrix

CountVectorizer team 2

In [391]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_products_counts)
X_train_tfidf.shape

(48, 168)

In [392]:
classlookup = None
filename = 'data/test_data_ml_classifier(1).csv'
with open(filename, newline='') as f:
                reader = csv.reader(f)
                try:
                    classlookup = defaultdict(list)
                    for key, val in reader:
                        classlookup[key] = val
                except csv.Error as e:
                    sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))

In [393]:
test_set_golden = list(classlookup.values())

In [394]:
test_set_golden = test_set_golden[1:]

In [395]:
test_set_golden = [int(i) for i in test_set_golden]

In [396]:
docs_test = list(classlookup.keys())[1:]

In [397]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [398]:
text_clf.fit(fullTrainingData, fullDataClassification)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [399]:
predicted = text_clf.predict(docs_test)
len(predicted)
len(test_set_golden)

50

In [400]:
np.mean(predicted == test_set_golden) # The Naive Bayes classifier has a precision of 79%

0.94

In [401]:
print(metrics.classification_report(test_set_golden, predicted,
                                    target_names=categories))

               precision    recall  f1-score   support

    canciones       0.92      0.96      0.94        25
documentacion       0.96      0.92      0.94        25

     accuracy                           0.94        50
    macro avg       0.94      0.94      0.94        50
 weighted avg       0.94      0.94      0.94        50



In [402]:
metrics.confusion_matrix(test_set_golden, predicted)

array([[24,  1],
       [ 2, 23]], dtype=int64)

In [403]:
values, counts = np.unique(test_set_golden, return_counts=True)

In [404]:
values, counts = np.unique(predicted, return_counts=True)

Try with SVM Support Vector Machine

IMPORTANT: works better with individual documents rather than a single one; but cleaning will use the whole documents.

In [405]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(fullTrainingData, fullDataClassification)
predicted = text_clf.predict(docs_test)
np.mean(predicted == test_set_golden)

0.96

In [406]:
print(metrics.classification_report(test_set_golden, predicted,
                                    target_names=categories))

               precision    recall  f1-score   support

    canciones       0.93      1.00      0.96        25
documentacion       1.00      0.92      0.96        25

     accuracy                           0.96        50
    macro avg       0.96      0.96      0.96        50
 weighted avg       0.96      0.96      0.96        50



In [407]:
metrics.confusion_matrix(test_set_golden, predicted)

array([[25,  0],
       [ 2, 23]], dtype=int64)